In [1]:
from ltlcross_runner import LtlcrossRunner
from IPython.display import display, Latex
import pandas as pd
import spot
spot.setup(show_default='.a')

In [2]:
def latexy(form_str,for_notebook=False):
    mode = 'sclatex' if for_notebook else 'latex'
    return '$' + spot.formula(form_str).to_str(mode) + '$'

In [3]:
def prepare_for_output(df):
    df.reset_index(inplace=True)
    df['form'] = pd.DataFrame.applymap(df.reset_index()[['formula']],latexy)
    df.drop('formula',1,inplace=True)
    df.drop('form_id',1,inplace=True)
    return df[['form'] + list(tool_order)]

If `rerun` is set to `False` the script uses the precomputed values. If set to `True`, all values are recomputed
(takes more time)

In [4]:
rerun = True

In [5]:
### Tools' setting ###
# PATHS
ltl2ldba  = 'ltl2ldba-0.9/bin/ltl2ldba %[MWi]f '
owl       = 'owl-1.0.0/bin/ltl2ldba %[MWi]f 2>/dev/null'
nba2ldba  = 'ltl2tgba --deterministic -f %f --ba | owl-1.0.0/bin/nba2ldba'
seminator = 'ltl2tgba --deterministic -f %f | ./seminator'
cy        = 'ltl2tgba --deterministic -f %f --ba | ./seminator --cy'
### Stripping command for ltl2ldba ###
# Ltl2ldba prints carbage collection messages to stdout
# which causes errors in ltlcross.
# We throw everything until the line HOA v1
awk  = ' | awk \'$0 == "HOA: v1" {i=1};i == 1\''

cut  = ' --cd'
simp_sd = ' | autfilt --small --tgba'
simp_cd = ' | autfilt -x simul=1,ba-simul=1 --small --tgba'
nos   = ' -s0' # disables Spot's simplifications used in Seminator
end  = ' > %O'

### Ltlcross runner configuration ###
tools = {## Cut-deterministic
#         'LTL2LDBA(cd)'      : ltl2ldba + awk + end,
#         'LTL2LDBA(cd)+spot' : ltl2ldba + awk + simp + end,
         'cd.ltl2ldba.no' : owl + awk + end,
         'cd.ltl2ldba.yes'    : owl + awk + simp_cd + end,
         'cd.seminator.no': seminator + nos + cut + end,
         'cd.seminator.yes'   : seminator + cut + end,
         'cd.cy.no'       : cy + nos + cut + end,     
         'cd.cy.yes'          : cy + cut + end,
         ### Semi-deterministic
         'sd.ltl2ldba.no' : owl + ' -n' + awk + end,
         'sd.ltl2ldba.yes'    : owl + ' -n' + awk + simp_sd + end,
         'sd.nba2ldba.no' : nba2ldba + awk + end,
         'sd.nba2ldba.yes'    : nba2ldba + awk + simp_sd + end,
         'sd.seminator.no': seminator + nos + end,
         'sd.seminator.yes'   : seminator + end,
         'sd.cy.no'       : cy + nos + end,  
         'sd.cy.yes'          : cy + end,
        }
sd_tools = ('sd.cy.no','sd.cy.yes',
            'sd.seminator.no','sd.seminator.yes',
            'sd.ltl2ldba.no','sd.ltl2ldba.yes',
            'sd.nba2ldba.no','sd.nba2ldba.yes')
cd_tools = ('cd.cy.no','cd.cy.yes',
            'cd.seminator.no','cd.seminator.yes',
            'cd.ltl2ldba.no','cd.ltl2ldba.yes')
### Formula files ###
det_files = ['formulae/literature_det.ltl']
nondet_files = ['formulae/literature_nondet.ltl']
det_r_files = ['formulae/rand_det.ltl']
nondet_r_files = ['formulae/rand_nondet.ltl']
### File with measured statistics ###
det_res_file = "sdba_literature_det.csv"
nondet_res_file = "sdba_literature_nondet.csv"
det_r_res_file = "sdba_random_det.csv"
nondet_r_res_file = "sdba_random_nondet.csv"
### Numbers to measure ###
cols = ['states','edges','transitions','nondet_states','exit_status','time','acc']
tool_order = sd_tools + cd_tools

In [6]:
runners = {}
for source in ('literature','random'):
    for t in ('det','cd','sd','nd'):
        name = '{}_{}'.format(source,t)
        runners[name] = \
            LtlcrossRunner(tools,\
                    res_filename='{}.csv'.format(name),\
                    formula_files=['formulae/{}.ltl'.format(name)],\
                    cols=cols)
        if rerun:
            runners[name].run_ltlcross(timeout='720',check=False)
        runners[name].parse_results()

## Cummulative results

In [7]:
for (name,r) in runners.items():
    print(name)
    display(r.cummulative())

random_sd


tool
cd.cy.no             750
cd.cy.yes            728
cd.ltl2ldba.no      1492
cd.ltl2ldba.yes     1277
cd.seminator.no      734
cd.seminator.yes     712
sd.cy.no             720
sd.cy.yes            720
sd.ltl2ldba.no      1228
sd.ltl2ldba.yes      787
sd.nba2ldba.no      1058
sd.nba2ldba.yes      720
sd.seminator.no      704
sd.seminator.yes     704
dtype: int64

literature_nd


tool
cd.cy.no            1020.0
cd.cy.yes            499.0
cd.ltl2ldba.no       420.0
cd.ltl2ldba.yes      360.0
cd.seminator.no      777.0
cd.seminator.yes     456.0
sd.cy.no             687.0
sd.cy.yes            418.0
sd.ltl2ldba.no       616.0
sd.ltl2ldba.yes      326.0
sd.nba2ldba.no       777.0
sd.nba2ldba.yes      459.0
sd.seminator.no      608.0
sd.seminator.yes     400.0
dtype: float64

random_nd


...

tool
cd.cy.no            13
cd.cy.yes           13
cd.ltl2ldba.no      61
cd.ltl2ldba.yes     40
cd.seminator.no     13
cd.seminator.yes    13
sd.cy.no            13
sd.cy.yes           13
sd.ltl2ldba.no      49
sd.ltl2ldba.yes     17
sd.nba2ldba.no      17
sd.nba2ldba.yes     13
sd.seminator.no     13
sd.seminator.yes    13
dtype: int64

random_det


tool
cd.cy.no            426
cd.cy.yes           426
cd.ltl2ldba.no      566
cd.ltl2ldba.yes     493
cd.seminator.no     413
cd.seminator.yes    413
sd.cy.no            426
sd.cy.yes           426
sd.ltl2ldba.no      664
sd.ltl2ldba.yes     442
sd.nba2ldba.no      521
sd.nba2ldba.yes     426
sd.seminator.no     413
sd.seminator.yes    413
dtype: int64

literature_det


tool
cd.cy.no             596
cd.cy.yes            596
cd.ltl2ldba.no      1033
cd.ltl2ldba.yes      806
cd.seminator.no      555
cd.seminator.yes     555
sd.cy.no             596
sd.cy.yes            596
sd.ltl2ldba.no      1263
sd.ltl2ldba.yes      851
sd.nba2ldba.no       807
sd.nba2ldba.yes      596
sd.seminator.no      555
sd.seminator.yes     555
dtype: int64

In [11]:
%mv literature* data
%mv random* data

In [9]:
for (name,r) in runners.items():
    print(name)
    display(r.compute_timeouts())

random_sd


tool
cd.cy.no            0
cd.cy.yes           0
cd.ltl2ldba.no      0
cd.ltl2ldba.yes     0
cd.seminator.no     0
cd.seminator.yes    0
sd.cy.no            0
sd.cy.yes           0
sd.ltl2ldba.no      0
sd.ltl2ldba.yes     0
sd.nba2ldba.no      0
sd.nba2ldba.yes     0
sd.seminator.no     0
sd.seminator.yes    0
dtype: int64

literature_nd


tool
cd.cy.no            0
cd.cy.yes           0
cd.ltl2ldba.no      0
cd.ltl2ldba.yes     0
cd.seminator.no     0
cd.seminator.yes    0
sd.cy.no            0
sd.cy.yes           1
sd.ltl2ldba.no      0
sd.ltl2ldba.yes     0
sd.nba2ldba.no      0
sd.nba2ldba.yes     1
sd.seminator.no     0
sd.seminator.yes    1
dtype: int64

random_nd


...

tool
cd.cy.no            0
cd.cy.yes           0
cd.ltl2ldba.no      0
cd.ltl2ldba.yes     0
cd.seminator.no     0
cd.seminator.yes    0
sd.cy.no            0
sd.cy.yes           0
sd.ltl2ldba.no      0
sd.ltl2ldba.yes     0
sd.nba2ldba.no      0
sd.nba2ldba.yes     0
sd.seminator.no     0
sd.seminator.yes    0
dtype: int64

random_det


tool
cd.cy.no            0
cd.cy.yes           0
cd.ltl2ldba.no      0
cd.ltl2ldba.yes     0
cd.seminator.no     0
cd.seminator.yes    0
sd.cy.no            0
sd.cy.yes           0
sd.ltl2ldba.no      0
sd.ltl2ldba.yes     0
sd.nba2ldba.no      0
sd.nba2ldba.yes     0
sd.seminator.no     0
sd.seminator.yes    0
dtype: int64

literature_det


tool
cd.cy.no            0
cd.cy.yes           0
cd.ltl2ldba.no      0
cd.ltl2ldba.yes     0
cd.seminator.no     0
cd.seminator.yes    0
sd.cy.no            0
sd.cy.yes           0
sd.ltl2ldba.no      0
sd.ltl2ldba.yes     0
sd.nba2ldba.no      0
sd.nba2ldba.yes     0
sd.seminator.no     0
sd.seminator.yes    0
dtype: int64

In [10]:
r = runners['literature_nd'].values
r.states[r.exit_status['sd.cy.yes'] != 'ok']

,tool,cd.cy.no,cd.cy.yes,cd.ltl2ldba.no,cd.ltl2ldba.yes,cd.seminator.no,cd.seminator.yes,sd.cy.no,sd.cy.yes,sd.ltl2ldba.no,sd.ltl2ldba.yes,sd.nba2ldba.no,sd.nba2ldba.yes,sd.seminator.no,sd.seminator.yes
form_id,formula,,,,,,,,,,,,,,
17,G((((a & Xa) | (!a & X!a)) & ((b & Xb) | (!b & X!b)) & ((c & Xc) | (!c & X!c)) & ((d & Xd) | (!d & X!d))) | X(!e & ((((a & Xa) | (!a & X!a)) & ((b & Xb) | (!b & X!b)) & ((c & Xc) | (!c & X!c)) & ((d & Xd) | (!d & X!d))) U e))),194.0,162.0,49.0,49.0,130.0,98.0,148.0,NaN,49.0,49.0,164.0,NaN,115.0,NaN
